<center><h1>Exploring the Competitors</h1></center>

### Part 1. Get location data using Foursquare

[Foursquare Places API](https://location.foursquare.com/products/places-api/) is very usefule online application used by many developers & other application like Uber etc. In this project you can used it to retrieve informtion about the places present in the neighborhoods of Toronto. The API returns a JSON file and we need to turn that into a data-frame. Here I’ve chosen similar businesses(pet grooming) for each neighborhood within a radius of 2.5km.

You will need to create an account with Foursquare to access the API.  It is free to sign up, and you get $200 free credit.



In [1]:
import requests
import pandas as pd

In [3]:
toronto_DF = pd.read_csv('Toronto_Neighborhoods.csv')

In [4]:
display(toronto_DF)

,Borough,Postalcode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160
...,...,...,...,...,...
98,York,M6C,Humewood-Cedarvale,43.693781,-79.428191
99,York,M6E,Caledonia-Fairbanks,43.689026,-79.453512
100,York,M6M,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
101,York,M6N,"Runnymede, The Junction North",43.673185,-79.487262


In [21]:
# Set up your API key and other constants for Foursquare API
API_KEY = 'fsq33xfBNISaHDapFiEhG4Ro8+1U4UZkzFo0RmJkXomhMAg='  # Replace with your actual Foursquare API key
LIMIT = 20  # Maximum number of venues to return
radius = 2500  # Search radius in meters

# Function to get nearby venues
def getNearbyVenues(names, postalcode, latitudes, longitudes, radius):
    venues_list = []

    for name, postalcode, lat, lng in zip(names, postalcode, latitudes, longitudes):
        print(f"Processing neighborhood: {name}")

        # Skip if latitude or longitude is missing
        if pd.isnull(lat) or pd.isnull(lng):
            print(f"Skipping {name} due to missing latitude or longitude.")
            continue

        # Create the API request URL and parameters
        url = 'https://api.foursquare.com/v3/places/search'
        params = {
            'll': f'{lat},{lng}',
            'radius': radius,
            'limit': LIMIT,
            'categories': 11095  # Pest control services category ID: search foursquare categories documentations
        }

        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }

        # Make the GET request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {name}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for venue in results.get('results', []):
            venue_name = venue.get('name')
            venue_id = venue.get('fsq_id')
            venue_location = venue.get('geocodes', {}).get('main', {})
            venue_lat = venue_location.get('latitude')
            venue_lng = venue_location.get('longitude')
            venue_categories = venue.get('categories', [])
            venue_category = venue_categories[0]['name'] if venue_categories else 'Unknown'

            # Append to the list
            venues_list.append([
                name,
                postalcode,
                lat,
                lng,
                venue_name,
                venue_id,
                venue_lat,
                venue_lng,
                venue_category
            ])

    # Create a DataFrame from the list
    nearby_venues = pd.DataFrame(venues_list, columns=[
        'Neighbourhood',
        'Postalcode',
        'Neighbourhood Latitude',
        'Neighbourhood Longitude',
        'Venue',
        'fsq_id',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ])

    return nearby_venues

# Example usage: Replace with the path to your Toronto Neighborhood CSV file
# Ensure 'toronto_DF' has the columns 'Neighbourhood', 'Postalcode', 'Latitude', and 'Longitude'
toronto_venues = getNearbyVenues(
    names=toronto_DF['Neighbourhood'],
    postalcode=toronto_DF['Postalcode'],
    latitudes=toronto_DF['Latitude'],
    longitudes=toronto_DF['Longitude'],
    radius=radius
)

# Display the first few rows of the resulting DataFrame
toronto_venues.head()

Processing neighborhood: Lawrence Park
Processing neighborhood: Davisville North
Processing neighborhood: North Toronto West
Processing neighborhood: Davisville
Processing neighborhood: Moore Park, Summerhill East
Processing neighborhood: Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Processing neighborhood: Roselawn
Processing neighborhood: Forest Hill North & West
Processing neighborhood: The Annex, North Midtown, Yorkville
Processing neighborhood: Rosedale
Processing neighborhood: St. James Town, Cabbagetown
Processing neighborhood: Church and Wellesley
Processing neighborhood: Regent Park, Harbourfront
Processing neighborhood: Garden District, Ryerson
Processing neighborhood: St. James Town
Processing neighborhood: Berczy Park
Processing neighborhood: Central Bay Street
Processing neighborhood: Richmond, Adelaide, King
Processing neighborhood: Harbourfront East, Union Station, Toronto Islands
Processing neighborhood: Toronto Dominion Centre, Design Exchange
Proc

,Neighbourhood,Postalcode,Neighbourhood Latitude,Neighbourhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,M4N,43.728020,-79.388790,Cain Pest & Wildlife Control,59bb5950112c6c44205c90af,43.728194,-79.402771,Pest Control Service
1,Lawrence Park,M4N,43.728020,-79.388790,AAA Professional Wildlife Control,049ad9a78fbc4226a4e3142d,43.710908,-79.393269,Pest Control Service
2,Lawrence Park,M4N,43.728020,-79.388790,Pest and Animal Control Toronto,52851516498ea2013e423f0b,43.711267,-79.396837,Pest Control Service
3,Lawrence Park,M4N,43.728020,-79.388790,AAA Affordable Wildlife Control,274729bde84d49dc9148401e,43.712275,-79.396109,Pest Control Service
4,Davisville North,M4P,43.712751,-79.390197,AAA Professional Wildlife Control,049ad9a78fbc4226a4e3142d,43.710908,-79.393269,Pest Control Service


In [22]:
# The total number of pest control services in Toronto
toronto_venues['Venue'].nunique()

137

In [7]:
# Oberserving the number of competitors in each neighbourhood
toronto_venues.groupby('Neighbourhood')['Venue'].count()

Neighbourhood
Agincourt                                           4
Bathurst Manor, Wilson Heights, Downsview North     2
Bayview Village                                     4
Bedford Park, Lawrence Manor East                   3
Berczy Park                                        17
                                                   ..
Willowdale) West                                    3
Willowdale, Newtonbrook                             5
Woodbine Heights                                   12
York Mills West                                     6
York Mills, Silver Hills                            4
Name: Venue, Length: 98, dtype: int64

### Part 2. Interactive leaflet map using coordinate data.


In [8]:
import folium # map rendering library

In [39]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

#Create map of all pest control services in Toronto
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="ln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 12)
# Add markers to map
for lat, lng, name, category, postcode in zip(toronto_venues['Venue Latitude'], toronto_venues['Venue Longitude'], toronto_venues['Venue'], toronto_venues['Venue Category'], toronto_venues['Postalcode']):
    label = '{}, {} - {}'.format(name, category, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
display(map_toronto)

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [37]:
# Pest control services in specific neighbourhood
target = 'Lawrence Park'

search_area = toronto_venues[toronto_venues['Neighbourhood'] == target]
latitude = toronto_DF[toronto_DF['Neighbourhood'] == target]['Latitude']
longitude = toronto_DF[toronto_DF['Neighbourhood'] == target]['Longitude']
display(search_area)
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area['Venue Latitude'], search_area['Venue Longitude'], search_area['Venue'], search_area['Neighbourhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

### Part 3. Customer Reviews
It seems like there is one one pet groomer withing 2km of Scarborough Village, I am interested in seeing customer's comments on Funny Bunny.  I can use [Foursquare's Place Tips API.](https://docs.foursquare.com/developer/reference/place-tips)


In [40]:
# a function to loop through the list of pest control services in the neighborhood and compile all the comments related to them
def getTips(venues, fsq_ids):
    venues_tips = []

    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue: {venue}")


        # Create the API request URL and parameters
        url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"


        # Make the GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {venue}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()
        
        # Parse the JSON response
        for tip in results:
            # Extract venue information
            tip_id = tip.get('id')
            tip_created = tip.get('created_at')
            tip_text = tip.get('text')
   

            # Append to the list
            venues_tips.append([
                venues,
                tip_id,
                tip_created,
                tip_text
            ])
            
    
    # Create a DataFrame from the list
    nearby_tips = pd.DataFrame(venues_tips, columns=[
        'venue_name',
        'id',
        'created_at',
        'text'
    ])

    return nearby_tips


nearby_tips = getTips(
    venues=search_area['Venue'],
    fsq_ids=search_area['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: Cain Pest & Wildlife Control
Processing venue: AAA Professional Wildlife Control
Processing venue: Pest and Animal Control Toronto
Processing venue: AAA Affordable Wildlife Control
Failed to get data for AAA Affordable Wildlife Control. Status code: 404
Error message: invalid place specified: 274729bde84d49dc9148401e


,venue_name,id,created_at,text


In [41]:
#Additionally feature...
from textblob import TextBlob

# Analyze sentiment for each tip
nearby_tips['sentiment'] = nearby_tips['text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Categorize sentiment as Positive, Neutral, or Negative
def categorize_sentiment(score):
    if score > 0.1:
        return 'Positive'
    elif score < -0.1:
        return 'Negative'
    else:
        return 'Neutral'

nearby_tips['sentiment_category'] = nearby_tips['sentiment'].apply(categorize_sentiment)

# Display results
print(nearby_tips[['venue_name', 'text', 'sentiment_category']].head())

Empty DataFrame
Columns: [venue_name, text, sentiment_category]
Index: []


### Now it is your turn to discover interesting search results using Foursquare
You are expected to read through the API documentation and customize the searches.  